In [1]:
import dask.dataframe as dd
import pandas as pd

In [2]:
data21 = dd.read_csv(r'훈련생 전달용\iMBANK_CUSTOMER_DATA(K-DigitalTraining)\iMBANK_CUSTOMER_DATA_2021(K-DigitalTraining).csv', encoding = 'cp949')
data22 = dd.read_csv(r'훈련생 전달용\iMBANK_CUSTOMER_DATA(K-DigitalTraining)\iMBANK_CUSTOMER_DATA_2022(K-DigitalTraining).csv', encoding = 'cp949')
data23 = dd.read_csv(r'훈련생 전달용\iMBANK_CUSTOMER_DATA(K-DigitalTraining)\iMBANK_CUSTOMER_DATA_2023(K-DigitalTraining).csv', encoding = 'cp949')

In [3]:
df4 = data21.compute()

In [8]:
df5 = data22.compute()

In [6]:
df6 = data23.compute()

In [4]:
df4.columns

Index(['기준년월', '고객ID', '연령대', '성별', '고객등급', '자택_시도', '자택_시군구', '수신_요구불예금',
       '수신_거치식예금', '수신_적립식예금', '수신_펀드', '수신_외화예금', '대출금액'],
      dtype='object')

In [5]:
df4 = df4[['고객ID','연령대']]

In [9]:
df5 = df5[['고객ID','연령대']]

In [7]:
df6 = df6[['고객ID','연령대']]

In [11]:
# 5에서 df6에 없는 고객ID 가져와서 df6에 추가
missing_in_df6_from_5 = df5[~df5['고객ID'].isin(df6['고객ID'])].copy()  # .copy()로 복사본 생성
df6_updated = pd.concat([df6, missing_in_df6_from_5], ignore_index=True)  # df6에 추가

# 4에서 df6 + df5에 없는 고객ID 가져와서 추가
missing_in_df6_and_5_from_4 = df4[~df4['고객ID'].isin(df6_updated['고객ID'])].copy()  # .copy()로 복사본 생성
df6_final = pd.concat([df6_updated, missing_in_df6_and_5_from_4], ignore_index=True)  # 최종 업데이트

# 출처 정보 추가
df6['source'] = 'df6'  # 기본 데이터에 출처 정보 추가
missing_in_df6_from_5.loc[:, 'source'] = 'df5'  # 복사본에 열 추가
missing_in_df6_and_5_from_4.loc[:, 'source'] = 'df4'  # 복사본에 열 추가

# 모든 데이터를 병합
df6_final = pd.concat([df6, missing_in_df6_from_5, missing_in_df6_and_5_from_4], ignore_index=True)

# 중복 제거 로직 (df6의 데이터를 우선 유지)
df6_final = df6_final.sort_values(by='source', key=lambda col: col.map({'df6': 0, 'df5': 1, 'df4': 2})).drop_duplicates(subset='고객ID', keep='first').reset_index(drop=True)

# 출처 정보 삭제 (원하지 않을 경우)
df6_final = df6_final.drop(columns=['source'])

In [13]:
df = df6_final
df

,고객ID,연령대
0,9999998161,60대이상
1,2972734318,10대미만
2,2972735594,60대이상
3,2972737061,60대이상
4,2972737585,30대
...,...,...
1999995,7795570273,50대
1999996,8661592297,50대
1999997,8378344508,50대
1999998,8403082734,50대


In [14]:
df['연령대'] = df['연령대'].astype(str).str[:2]  # 앞 두 글자 추출
df['연령대'] = pd.to_numeric(df['연령대'], errors='coerce').astype('Int64')  # 숫자로 변환 및 결측값 처리

In [15]:
df

,고객ID,연령대
0,9999998161,60
1,2972734318,10
2,2972735594,60
3,2972737061,60
4,2972737585,30
...,...,...
1999995,7795570273,50
1999996,8661592297,50
1999997,8378344508,50
1999998,8403082734,50


In [16]:
df.to_csv('age.csv', encoding='utf-8', index=False)